# 3: Complex wind
-----------------

In this exercise we look at the complex wind of $\pi^{1}\text{Gru}$.

In [1]:
import numpy             as np
import matplotlib.pyplot as plt
import magritte.tools    as tools
import magritte.plot     as plot
import magritte.setup    as setup
import magritte.core     as magritte
import magritte.mesher   as mesher

from astropy       import units, constants
from scipy.spatial import Delaunay

In [2]:
# Define names for the data files
model_file = 'output/magritte_complex_wind.hdf5'
lamda_file = 'data/co.txt'

If necessary, download the line data file form the [LAMDA database](https://home.strw.leidenuniv.nl/~moldata/).

In [3]:
!wget "https://home.strw.leidenuniv.nl/~moldata/datafiles/co.dat" --output-document $lamda_file

--2024-02-13 12:47:52--  https://home.strw.leidenuniv.nl/~moldata/datafiles/co.dat
Resolving home.strw.leidenuniv.nl (home.strw.leidenuniv.nl)... 132.229.214.179
Connecting to home.strw.leidenuniv.nl (home.strw.leidenuniv.nl)|132.229.214.179|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 444204 (434K)
Saving to: ‘data/co.txt’

data/co.txt         100%[===================>] 433.79K  --.-KB/s    in 0.07s   

2024-02-13 12:47:52 (5.81 MB/s) - ‘data/co.txt’ saved [444204/444204]



In [4]:
# Define model parameters
r_in  = (  65.0 * units.au        ).si.value  # inner boundary of the model
r_out = (3300.0 * units.au        ).si.value  # outer boundary of the model
R1    = (2000.0 * units.au        ).si.value  # outer radius of the torus
R3    = (3300.0 * units.au        ).si.value  # outer radius of the bipolar wind
d     = ( 270.0 * units.au        ).si.value  # inner diameter of the torus
h     = ( 130.0 * units.au        ).si.value  # h/2 = starting radius of the bipolar wind
φ0    = (  90.0 * units.deg       ).si.value  # 2φ0 = opening angle of the flared disk torus
v1a   = (   8.0 * units.km/units.s).si.value  # constant for the velocity profile of the torus
v1b   = (   5.0 * units.km/units.s).si.value  # constant for the velocity profile of the torus
v3a   = (  11.0 * units.km/units.s).si.value  # constant for the velocity profile of the bipolar outflow
v3b   = (  89.0 * units.km/units.s).si.value  # constant for the velocity profile of the bipolar outflow
T0    = ( 190.0 * units.K         ).si.value  # initial temperature at r = 1e15 cm = h/2
n1a   = (9.0e+7 * units.cm**(-3)  ).si.value  # scaling factor for the density profile of the torus
n3a   = (6.0e+7 * units.cm**(-3)  ).si.value  # scaling factor for the density profile of the bipolar wind
fφ0   =  2.5                                  # value of fφ at the top and bottom edges of the torus
α     =  0.0                                  # -3+α = exponent of the density profile of the bipolar wind
β     = -0.15                                 # -0.7+β = exponent of the temperature profile

X_CO   =  6.5e-4   # fractional abundance of CO relative to H2 (Knapp et al. 1999)
v_turb = (2.0 * units.km/units.s).si.value    # turbulent velocity


def cartesian_to_spherical(x, y, z):
    """
    Convert cartesian to spherical coordinates.
    """
    r = np.sqrt(x**2 + y**2 + z**2)
    φ = np.pi/2 - np.arccos(z / r)
    θ = np.arctan2(y, x)
    return (r, φ, θ)


def fφ(φ):
    """
    Linear function, used to define the torus shape (Doan et al. 2017).
    """
    return 1.0 + np.abs(φ/φ0) * fφ0 


def mask_1(r, φ, θ):
    """
    Mask for component 1: torus.
    """
    return np.logical_and(r > d/2, r < R1, np.abs(φ) < φ0)


def mask_3(r, φ, θ):
    """
    Mask for component 3: bipolar outflow.
    """
    return np.logical_and(r > h/2, r < R3, np.abs(φ) > φ0)


def number_density_H2(x, y, z):
    """
    H2 number density distributions
    """
    r, φ, θ = cartesian_to_spherical(x, y, z)
    # Get the masks for the different components
    m1 = mask_1(r, φ, θ)
    m3 = mask_3(r, φ, θ)
    # Initialize the number density array
    nH2 = np.zeros_like(r)
    # H2 number density distribution of the torus
    n1 = n1a * (r / (1.0e+15 * units.cm).si.value)**(-3) * 1.0 / fφ(φ)
    # H2 number density distribution of the bipolar outflow
    n3 = n3a * (r / (1.0e+15 * units.cm).si.value)**(-3+α)
    # Apply masks
    nH2[m1] = n1[m1]
    # nH2[m3] = n3[m3]
    # Return the H2 number density
    return nH2


def velocity(x, y, z):
    """
    Velocity distributions
    """
    r, φ, θ = cartesian_to_spherical(x, y, z)
    # Get the masks for the different components
    m1 = mask_1(r, φ, θ)
    m3 = mask_3(r, φ, θ)
    # Initialize the velocity array
    vr = np.zeros_like(r)
    # Radial velocity of component one, Eq. (1) in Doan et al. (2017)
    v1 = (v1a + v1b * r / R1) * fφ(φ)
    # Radial velocity of component one, Eq. (2) in Doan et al. (2017)
    v3 =  v3a + v3b * z / R3
    # Apply masks
    vr[m1] = v1[m1]
    vr[m3] = v3[m3]
    # Return the three cartesian components of the velocity vector
    return np.array((vr*x/r, vr*y/r, vr*z/r)).T


def temperature(x, y, z):
    """
    Temperature distribution.
    """
    r, φ, θ = cartesian_to_spherical(x, y, z)
    # Temperature distribution
    return T0 * (r / (1.0e+15 * units.cm).si.value)**(-0.7+β)

Create a point cloud for the model.

In [5]:
resolution = 75

# Create coordinate axis for the background mesh
xs = np.linspace(-r_out, +r_out, resolution, endpoint=True)
ys = np.linspace(-r_out, +r_out, resolution, endpoint=True)
zs = np.linspace(-r_out, +r_out, resolution, endpoint=True)

# Create a background mesh
(Xs, Ys, Zs) = np.meshgrid(xs, ys, zs)

# Extract positions of points in background mesh
position = np.array((Xs.ravel(), Ys.ravel(), Zs.ravel())).T

# Evaluate a tracer (here the H2 number density density) on the background mesh
tracer      = number_density_H2(Xs, Ys, Zs)
tracer_min  = np.min(tracer[tracer!=0.0])   # Find smallest non-zero value
tracer     += tracer_min                    # add smallest non-zero value to avoid zeros

# Create a point cloud for the model
positions_reduced, nb_boundary = mesher.remesh_point_cloud(
    positions = position,
    data      = tracer.ravel(),
    max_depth = 10,
    threshold = 2.0e-1,
    hullorder = 4
)

# Add inner boundary to the model
positions_reduced, nb_boundary = mesher.point_cloud_add_spherical_inner_boundary(
    remeshed_positions = positions_reduced,
    nb_boundary        = nb_boundary,
    radius             = r_in,
    healpy_order       = 5,
    origin             = np.array([0.0, 0.0, 0.0]).T
)

# Add outer boundary to the model
positions_reduced, nb_boundary = mesher.point_cloud_add_spherical_outer_boundary(
    remeshed_positions = positions_reduced,
    nb_boundary        = nb_boundary,
    radius             = r_out,
    healpy_order       = 15,
    origin             = np.array([0.0, 0.0, 0.0]).T
)

# Numbenr of points in the model
npoints = len(positions_reduced)

# Extract Delaunay vertices (= Voronoi neighbors)
delaunay = Delaunay(positions_reduced)
(indptr, indices) = delaunay.vertex_neighbor_vertices
neighbors = [indices[indptr[k]:indptr[k+1]] for k in range(npoints)]
nbs       = [n for sublist in neighbors for n in sublist]
n_nbs     = [len(sublist) for sublist in neighbors]

new interior points:  22310
number boundary points:  1538


In [6]:
position = positions_reduced
velocity = velocity         (position[:,0], position[:,1], position[:,2])
tmp      = temperature      (position[:,0], position[:,1], position[:,2])
nH2      = number_density_H2(position[:,0], position[:,1], position[:,2])
nCO      = X_CO * nH2
trb      = v_turb * np.ones (npoints)

In [8]:
model = magritte.Model ()                      # Create model object

model.parameters.set_model_name (model_file)   # Magritte model file
model.parameters.set_dimension  (3)            # This is a 3D model
model.parameters.set_npoints    (npoints)      # Number of points
model.parameters.set_nrays      (12)           # Number of rays
model.parameters.set_nspecs     (3)            # Number of species (min. 5)
model.parameters.set_nlspecs    (1)            # Number of line species
model.parameters.set_nquads     (5)            # Number of quadrature points

model.geometry.points.position.set(position)
model.geometry.points.velocity.set(velocity / constants.c.si.value)   # velocities w.r.t. speed of light 

model.geometry.points.  neighbors.set(  nbs)
model.geometry.points.n_neighbors.set(n_nbs)

model.chemistry.species.abundance = np.array((nCO, nH2, np.zeros(npoints))).T
model.chemistry.species.symbol    = ['CO', 'H2', 'e-']

model.thermodynamics.temperature.gas  .set( tmp                         )
model.thermodynamics.turbulence.vturb2.set((trb/constants.c.si.value)**2)

model.parameters.set_nboundary(nb_boundary)
model.geometry.boundary.boundary2point.set(np.arange(nb_boundary))

# direction = np.array([[+1,0,0], [-1,0,0]])            # Comment out to use all directions
# model.geometry.rays.direction.set(direction)          # Comment out to use all directions
# model.geometry.rays.weight   .set(0.5 * np.ones(2))   # Comment out to use all directions

model = setup.set_uniform_rays            (model)   # Uncomment to use all directions
model = setup.set_boundary_condition_CMB  (model)
# model = setup.set_linedata_from_LAMDA_file(model, lamda_file, {'considered transitions': [0]})
model = setup.set_linedata_from_LAMDA_file(model, lamda_file)   # Consider all transitions
model = setup.set_quadrature              (model)

# Write and read model to initialize all variables
model.write()
model.read ()

Writing parameters...
Writing points...
Writing rays...
Writing boundary...
Writing chemistry...
Writing species...
Writing thermodynamics...
Writing temperature...
Writing turbulence...
Writing lines...
Writing lineProducingSpecies...
Writing linedata...
ncolpoar = 2
--- colpoar = 0
Writing collisionPartner...
(l, c) = 0, 0
--- colpoar = 1
Writing collisionPartner...
(l, c) = 0, 1
Writing quadrature...
Writing populations...
Writing radiation...
Writing frequencies...
                                           
-------------------------------------------
  Reading Model...                         
-------------------------------------------
 model file = output/magritte_complex_wind.hdf5
-------------------------------------------
Reading parameters...
Reading points...
Reading rays...
Reading boundary...
Reading chemistry...
Reading species...
Reading thermodynamics...
Reading temperature...
Reading turbulence...
Reading lines...
Reading lineProducingSpecies...
Reading linedata...
re

In [9]:
# Initialize model with LTE level populations
model.compute_spectral_discretisation ()
model.compute_inverse_line_widths     ()
model.compute_LTE_level_populations   ()

Computing spectral discretisation...


0

Computing inverse line widths...
Computing LTE level populations...


In [10]:
%%time

# Iterate level populations until statistical equilibrium
model.compute_level_populations_sparse (True, 20)

Starting iteration 0
CPU times: user 28min 16s, sys: 16.5 s, total: 28min 32s
Wall time: 28min 25s
using ng acceleration? 0
Computing the radiation field...
Computing radiation field...
--- rr = 0
--- rr = 1
--- rr = 2
--- rr = 3
--- rr = 4
--- rr = 5
Tot | Compute Radiation Field : 84.628134 seconds
Tot | Compute Statistical Equilibrium : 0.964753 seconds
Already 83.8666 % converged!
Starting iteration 1
using ng acceleration? 0
Computing the radiation field...
Computing radiation field...
--- rr = 0
--- rr = 1
--- rr = 2
--- rr = 3
--- rr = 4
--- rr = 5
Tot | Compute Radiation Field : 167.882517 seconds
Tot | Compute Statistical Equilibrium : 1.888642 seconds
Already 83.941 % converged!
Starting iteration 2
using ng acceleration? 0
Computing the radiation field...
Computing radiation field...
--- rr = 0
--- rr = 1
--- rr = 2
--- rr = 3
--- rr = 4
--- rr = 5
Tot | Compute Radiation Field : 253.081618 seconds
Tot | Compute Statistical Equilibrium : 2.807361 seconds
Already 83.9629 % co

20

ion? 1
Ng acceleration max order: 3 used order: 2
Already 87.4039 % converged!
Starting iteration 6
using ng acceleration? 0
Computing the radiation field...
Computing radiation field...
--- rr = 0
--- rr = 1
--- rr = 2
--- rr = 3
--- rr = 4
--- rr = 5
Tot | Compute Radiation Field : 583.206821 seconds
Tot | Compute Statistical Equilibrium : 6.318240 seconds
Already 89.4783 % converged!
Starting iteration 7
using ng acceleration? 0
Computing the radiation field...
Computing radiation field...
--- rr = 0
--- rr = 1
--- rr = 2
--- rr = 3
--- rr = 4
--- rr = 5
Tot | Compute Radiation Field : 666.634033 seconds
Tot | Compute Statistical Equilibrium : 7.178121 seconds
Already 93.9211 % converged!
Starting iteration 8
using ng acceleration? 0
Computing the radiation field...
Computing radiation field...
--- rr = 0
--- rr = 1
--- rr = 2
--- rr = 3
--- rr = 4
--- rr = 5
Tot | Compute Radiation Field : 751.224454 seconds
Tot | Compute Statistical Equilibrium : 8.096218 seconds
Already 95.4524 %

In [11]:
fcen = model.lines.lineProducingSpecies[0].linedata.frequency[0]
vpix = 1.0e+3   # velocity pixel size [m/s]
dd   = vpix * (model.parameters.nfreqs()-1)/2 / magritte.CC
fmin = fcen - fcen*dd
fmax = fcen + fcen*dd

# Ray orthogonal to image plane
ray_nr = 0

model.compute_spectral_discretisation (fmin, fmax)#bins the frequency spectrum [fmin, fmax] into model.parameters.nfreqs bins.
# model.compute_spectral_discretisation (fmin, fmax, 31)#bins using the specified amount of frequency bins (31). Can be any integer >=1

model.compute_image_new               (ray_nr, 512, 512)#using a resolution of 512x512 for the image.
#Instead of definining a ray index [0, nrays-1], you can also define a ray direction for the imager
#model.compute_image_new              (rx, ry, rz, 512, 512)#in which (rx, ry, rz) is the (normalized) ray direction

Computing spectral discretisation...
Computing image new...


0

In [12]:
plot.image_mpl(
    model,
    image_nr =  -1,
    zoom     = 1.3,
    npix_x   = 256,
    npix_y   = 256,
    x_unit   = units.au,
    v_unit   = units.km / units.s)

Created image directory: /Users/sofiaw/git/StellarAtmospheres_assignments/ex3_complex_wind/output/images/


100%|████████████████████████████████████████████████████████████████████████████████| 200/200 [03:38<00:00,  1.09s/it]


interactive(children=(IntSlider(value=99, description='v', max=199), Output()), _dom_classes=('widget-interact…

<function magritte.plot.image_mpl.<locals>.<lambda>(v)>

In [13]:
import k3d
from k3d.colormaps import matplotlib_color_maps
from k3d.helpers import map_colors
from numpy.linalg import norm

p = np.linspace(-1, 1, 10)

def f(x, y, z):
    return y * z, x * z, x * y

vectors = np.array([[[f(x, y, z) for x in p] for y in p] for z in p]).astype(np.float32)
norms = np.apply_along_axis(norm, 1, vectors.reshape(-1, 3))

plt_vector_field = k3d.vector_field(vectors,
                                    head_size=1.5,
                                    scale=2,
                                    bounds=[-1, 1, -1, 1, -1, 1])

colors = map_colors(norms, matplotlib_color_maps.Turbo, [0, 1]).astype(np.uint32)
plt_vector_field.colors = np.repeat(colors, 2)

plot = k3d.plot()
plot += plt_vector_field
plot.display()

Output()

In [14]:
data = np.log(rhos)

plt_volume = k3d.volume(
    volume      = data.astype(np.float32),
    alpha_coef  = 5.0,
    color_range = [0, 30]
)


plot  = k3d.plot()
plot += plt_volume
plot.display()

NameError: name 'rhos' is not defined

In [28]:
from ipywidgets import interact

In [29]:
data = np.log(rhos)

def plot(i):
    plt.figure(dpi=150)
    plt.imshow(data[:,i,:], vmin=data.min(), vmax=data.max())

interact(plot, i=(0,len(rhos)))

interactive(children=(IntSlider(value=37, description='i', max=75), Output()), _dom_classes=('widget-interact'…

<function __main__.plot(i)>

<Figure size 900x600 with 0 Axes>

<Figure size 900x600 with 0 Axes>